In [ ]:
import torch
import torchvision
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
!pip install openai-clip
import clip
import os
import pandas as pd
import itertools
import numpy as np
import cv2
import json
from itertools import chain
import random
import requests
from io import BytesIO
from PIL import Image
from urllib.request import urlopen
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 8.0 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=d2f1f6fe92ec1c7ce57bae995be862b1712cc303faa54bdad01a90c0b87ed418
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip


In [ ]:
tags=['Architectural Landscape','Buildings','Houses','Cosmic Bodies','Marine','Natural Landscape','Natural Patterns','Nonbio','Animals','Humans','Plants & Trees','Water','Seascape','seasonal-and-natural-phenomenon','Still-life']

text_path='/content/Pictures.txt'
with open(text_path, 'r') as file:
    files_list = file.readlines()

In [ ]:
class Decoder(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.batch_norm1 = nn.BatchNorm1d(256)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 128)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(128, output_dim)
        self.sigmoid=nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [ ]:
class Season(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Season, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.batch_norm1 = nn.BatchNorm1d(256)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(256, 128)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        self.fc3 = nn.Linear(128, output_dim)
        self.sigmoid=nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.batch_norm1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.batch_norm2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
input_dim = 512
output_dim = 15



100%|███████████████████████████████████████| 338M/338M [00:05<00:00, 69.2MiB/s]


In [ ]:
def get_string(arr,tags):
  bio_tags=['Autumn','Natural Phenomena','Northern Lights','Stormy Weather','Winter','Cosmic Bodies','Marine','Natural Landscape','Natural Patterns','Animals','Plants & Trees','Water','Seascape','Still-life']


  if arr[0]==1:
    arr[5]=0
  if arr[4]==1:
    arr[5]=0
  if arr[12]==1:
    arr[5]=0

  if arr[7]==1 and sum(arr)>1:
    arr[7]=0
  if sum(arr)==0:
    arr[7]=1


  label=''
  for i in range(15):
    if arr[i]==1:
      label=label+tags[i]+', '
  label=label[:-2]

  if label=='Nonbio':
    label=''

  return label

In [ ]:
def check_seasons(image_features):
  classes_str=['Autumn','Natural Phenomena','Northern Lights','Stormy Weather','Winter']
  season=Season(input_dim, 5).to(device)
  season.load_state_dict(torch.load('Seasons.pth', map_location=device))
  season.eval()
  outputs2 = season(image_features)
  pred2 = outputs2.argmax(dim=1, keepdim=True)
  pred2 = pred2.cpu().tolist()
  pred2=list(chain.from_iterable(pred2))

  t=classes_str[pred2[0]]
  return t

In [ ]:
def get_label(image):
  bio_tags=['Autumn','Natural Phenomena','Northern Lights','Stormy Weather','Winter','Architectural Landscape','Cosmic Bodies','Marine','Natural Landscape','Natural Patterns','Animals','Plants & Trees','Water','Seascape','Still-life']


  transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
  ])


  decoder=Decoder(input_dim, output_dim).to(device)
  decoder.load_state_dict(torch.load('Bio.pth', map_location=device))

  image = transform(image)
  image = image.unsqueeze(0)
  image=image.to(device)
  decoder.eval()
  with torch.no_grad():
    image_features = model.encode_image(image)
  image_features=image_features.to(torch.float32)
  outputs = decoder(image_features)
  pred = (outputs > 0.70).float()
  pred = pred.cpu().tolist()
  pred=list(chain.from_iterable(pred))

  d=pred.index(max(pred))
  pt=0
  if pred[13]==1:
    pt=check_seasons(image_features)
    pred[13]=0

  if d==13:
    dom=pt
  else:
    dom=tags[d]
  predicted_tag=get_string(pred,tags)
  if pt!=0:
    predicted_tag+=','+pt

  bio=0
  for b in bio_tags:
    if b in predicted_tag:
      bio=1
      break

  if predicted_tag=='':
    dom='Nonbio'

  print(pred,predicted_tag)

  return predicted_tag,bio, dom





In [ ]:
def get_traits():
  traits=[]
  Bio=[]
  dom=[]
  for f in files_list:
    resp = urlopen(f)
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    img_rgb=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    data=Image.fromarray(img_rgb)

    t,b,d=get_label(data)

    traits.append(t)
    Bio.append(b)
    dom.append(d)
  return traits,Bio,dom

In [ ]:
traits,Bio,dom=get_traits()


[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0] Natural Landscape, Plants & Trees
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0] Natural Landscape, Animals, Humans, Plants & Trees, Water
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] Natural Landscape
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] Humans
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] Humans
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] Humans
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] Humans
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] Humans
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0] Humans
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0, 0.0] Natural Landsca

In [ ]:
df=pd.DataFrame({'Image':files_list,'Biophilic traits':traits,'Dominant trait':dom,'Biophilic Flag':Bio},columns=['Image','Biophilic traits','Dominant trait','Biophilic Flag'])
df.head(10)

,Image,Biophilic traits,Dominant trait,Biophilic Flag
0,http://api.vieunite.com/ng/files/1667124136509...,"Natural Landscape, Plants & Trees",Natural Landscape,1
1,http://api.vieunite.com/ng/files/1667124136509...,"Natural Landscape, Animals, Humans, Plants & T...",Natural Landscape,1
2,http://api.vieunite.com/ng/files/1667124136509...,Natural Landscape,Natural Landscape,1
3,http://api.vieunite.com/ng/files/1667124136509...,Humans,Humans,0
4,http://api.vieunite.com/ng/files/1667124136509...,Humans,Humans,0
5,http://api.vieunite.com/ng/files/1667124136509...,Humans,Humans,0
6,http://api.vieunite.com/ng/files/1667124136509...,,Nonbio,0
7,http://api.vieunite.com/ng/files/1579488114120...,Humans,Humans,0
8,http://api.vieunite.com/ng/files/1667124136509...,Humans,Humans,0
9,http://api.vieunite.com/ng/files/1667124136509...,Humans,Humans,0


In [ ]:
df.to_csv('Biophilic1.csv', index = False)